# Import libraries necessary for this project

In [1]:
import numpy as np
import pandas as pd
import xgboost
from sklearn.metrics import accuracy_score
from sklearn.metrics import auc
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split

# Loading data 

In [2]:
data= pd.read_csv("C:\\Users\\Sumit Ranjan\\Desktop\\pgdm\\semester 5\\loan.csv")

C:\Users\Sumit Ranjan\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3020: DtypeWarning: Columns (19,47,55,112,123,124,125,128,129,130,133,139,140,141) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


# Introduction To The Data

In [3]:
data.head()

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,...,hardship_payoff_balance_amount,hardship_last_payment_amount,disbursement_method,debt_settlement_flag,debt_settlement_flag_date,settlement_status,settlement_date,settlement_amount,settlement_percentage,settlement_term
0,NaN,NaN,2500,2500,2500.0,36 months,13.56,84.92,C,C1,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,30000,30000,30000.0,60 months,18.94,777.23,D,D2,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,5000,5000,5000.0,36 months,17.97,180.69,D,D1,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,4000,4000,4000.0,36 months,18.94,146.51,D,D2,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,30000,30000,30000.0,60 months,16.14,731.78,C,C4,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
new_data = data[['member_id', 'loan_amnt', 'funded_amnt', 'addr_state', 'funded_amnt_inv', 'grade', 'term', 'emp_length', 'int_rate', 'annual_inc', 'dti', 'delinq_2yrs', 'inq_last_6mths', 'mths_since_last_delinq', 'mths_since_last_record', 'open_acc', 'pub_rec', 'revol_bal', 'revol_util', 'total_acc', 'total_rec_int', 'total_rec_late_fee', 'recoveries', 'collection_recovery_fee', 'collections_12_mths_ex_med', 'mths_since_last_major_derog', 'acc_now_delinq', 'tot_coll_amt', 'tot_cur_bal', 'total_rev_hi_lim', 'loan_status']]

In [5]:
new_data.head()

,member_id,loan_amnt,funded_amnt,addr_state,funded_amnt_inv,grade,term,emp_length,int_rate,annual_inc,...,total_rec_late_fee,recoveries,collection_recovery_fee,collections_12_mths_ex_med,mths_since_last_major_derog,acc_now_delinq,tot_coll_amt,tot_cur_bal,total_rev_hi_lim,loan_status
0,NaN,2500,2500,NY,2500.0,C,36 months,10+ years,13.56,55000.0,...,0.0,0.0,0.0,0.0,NaN,0.0,0.0,16901.0,42000.0,Current
1,NaN,30000,30000,LA,30000.0,D,60 months,10+ years,18.94,90000.0,...,0.0,0.0,0.0,0.0,NaN,0.0,1208.0,321915.0,50800.0,Current
2,NaN,5000,5000,MI,5000.0,D,36 months,6 years,17.97,59280.0,...,0.0,0.0,0.0,0.0,NaN,0.0,0.0,110299.0,24100.0,Current
3,NaN,4000,4000,WA,4000.0,D,36 months,10+ years,18.94,92000.0,...,0.0,0.0,0.0,0.0,NaN,0.0,686.0,305049.0,7000.0,Current
4,NaN,30000,30000,MD,30000.0,C,60 months,10+ years,16.14,57250.0,...,0.0,0.0,0.0,0.0,NaN,0.0,0.0,116007.0,23100.0,Current


# Data Cleaning

In [6]:
new_data['term'].replace(to_replace=' months', value='', regex=True, inplace=True)
new_data['term'] = pd.to_numeric(new_data['term'], errors='coerce')

C:\Users\Sumit Ranjan\Anaconda3\lib\site-packages\pandas\core\generic.py:5890: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)
C:\Users\Sumit Ranjan\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [7]:
new_data['emp_length'].replace('n/a', '0', inplace=True)
new_data['emp_length'].replace(to_replace='\+ years', value='', regex=True, inplace=True)
new_data['emp_length'].replace(to_replace=' years', value='', regex=True, inplace=True)
new_data['emp_length'].replace(to_replace='< 1 year', value='0', regex=True, inplace=True)
new_data['emp_length'].replace(to_replace=' year', value='', regex=True, inplace=True)
new_data['emp_length'] = pd.to_numeric(new_data['emp_length'], errors='coerce')

C:\Users\Sumit Ranjan\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [8]:
new_data['grade'].replace(to_replace='A', value='0', regex=True, inplace=True)
new_data['grade'].replace(to_replace='B', value='1', regex=True, inplace=True)
new_data['grade'].replace(to_replace='C', value='2', regex=True, inplace=True)
new_data['grade'].replace(to_replace='D', value='3', regex=True, inplace=True)
new_data['grade'].replace(to_replace='E', value='4', regex=True, inplace=True)
new_data['grade'].replace(to_replace='F', value='5', regex=True, inplace=True)
new_data['grade'].replace(to_replace='G', value='6', regex=True, inplace=True)
new_data['grade'] = pd.to_numeric(new_data['grade'], errors='coerce')

C:\Users\Sumit Ranjan\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [9]:
new_data.isnull().sum().sort_values(ascending=False).head()

member_id                      2260668
mths_since_last_record         1901512
mths_since_last_major_derog    1679893
mths_since_last_delinq         1158502
emp_length                      146907
dtype: int64

In [10]:
cols = ['term', 'loan_amnt', 'funded_amnt', 'int_rate', 'grade', 'annual_inc', 'dti', 'mths_since_last_delinq', 'mths_since_last_record', 'open_acc', 'revol_bal', 'revol_util', 'total_acc', 'total_rec_int', 'mths_since_last_major_derog', 'tot_coll_amt', 'tot_cur_bal', 'total_rev_hi_lim', 'emp_length']
for col in cols:
    new_data[col].fillna(new_data[col].median(), inplace=True)

C:\Users\Sumit Ranjan\Anaconda3\lib\site-packages\pandas\core\generic.py:5434: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)


In [11]:
cols = ['acc_now_delinq', 'total_rec_late_fee', 'recoveries', 'collection_recovery_fee', 'collections_12_mths_ex_med']
for col in cols:
    new_data[col].fillna(0, inplace=True)

#  Loan status columns into two categories i.e. default and fully paid

In [12]:
default=['Charged Off','Late (31-120 days)','Default','Late (16-30 days)','In Grace Period']
paid=['Fully Paid']
new_data['loan_status']=new_data['loan_status'].apply(lambda x: 'Default' if x in default else x)
new_data['loan_status']=new_data['loan_status'].apply(lambda x: 'Fully Paid' if x in paid else x)
new_data=new_data[(new_data['loan_status']=='Default') | (new_data['loan_status']=='Fully Paid')]
target_variable=new_data['loan_status'].apply(lambda x: 1 if x=='Default' else 0)
new_data.drop('loan_status',axis=1,inplace=True)

C:\Users\Sumit Ranjan\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\Sumit Ranjan\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


In [13]:
selected_cols = ['member_id', 'emp_length', 'loan_amnt', 'funded_amnt', 'funded_amnt_inv', 'grade', 'int_rate', 'annual_inc', 'dti', 'mths_since_last_delinq', 'mths_since_last_record', 'open_acc', 'revol_bal', 'revol_util', 'total_acc', 'total_rec_int', 'total_rec_late_fee', 'mths_since_last_major_derog', 'tot_cur_bal', 'total_rev_hi_lim', 'tot_coll_amt', 'recoveries', 'collection_recovery_fee', 'term', 'acc_now_delinq', 'collections_12_mths_ex_med']
new_data_2=new_data[selected_cols]

# Creating new variable for better training of data

# Loan to income ratio

In [14]:
new_data_2['loan_to_income_ratio'] = new_data_2['annual_inc']/new_data_2['funded_amnt_inv']

C:\Users\Sumit Ranjan\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [15]:
new_data_2.head()

,member_id,emp_length,loan_amnt,funded_amnt,funded_amnt_inv,grade,int_rate,annual_inc,dti,mths_since_last_delinq,...,mths_since_last_major_derog,tot_cur_bal,total_rev_hi_lim,tot_coll_amt,recoveries,collection_recovery_fee,term,acc_now_delinq,collections_12_mths_ex_med,loan_to_income_ratio
100,NaN,5.0,30000,30000,30000.0,3,22.35,100000.0,30.46,51.0,...,44.0,472330.0,42200.0,0.0,0.0,0.0,36,0.0,0.0,3.333333
152,NaN,0.0,40000,40000,40000.0,2,16.14,45000.0,50.53,31.0,...,44.0,271068.0,54200.0,0.0,0.0,0.0,60,0.0,0.0,1.125000
170,NaN,10.0,20000,20000,20000.0,0,7.56,100000.0,18.92,48.0,...,44.0,515779.0,85100.0,0.0,0.0,0.0,36,0.0,0.0,5.000000
186,NaN,10.0,4500,4500,4500.0,1,11.31,38500.0,4.64,25.0,...,25.0,29137.0,29200.0,306.0,0.0,0.0,36,0.0,0.0,8.555556
215,NaN,3.0,8425,8425,8425.0,4,27.27,450000.0,12.37,31.0,...,44.0,690320.0,56000.0,0.0,0.0,0.0,36,0.0,0.0,53.412463


In [16]:
new_data_2['avl_lines'] = new_data_2['total_acc'] - new_data_2['open_acc']

C:\Users\Sumit Ranjan\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


# Total Interest paid

In [17]:
new_data_2['int_paid'] = new_data_2['total_rec_int'] + new_data_2['total_rec_late_fee']

C:\Users\Sumit Ranjan\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


# Total repayment 

In [18]:
new_data_2['total_repayment'] =  ((new_data_2['recoveries']/new_data_2['funded_amnt_inv']) * 100)

In [19]:
new_data_2.shape

(1338224, 30)

# Splitting the data as train and test

In [24]:
X_train,X_test,y_train,y_test = train_test_split(np.array(new_data_2),target_variable,test_size=.30,random_state=1) 
X_train.shape
X_test.shape

(401468, 30)

In [27]:
eval_set=[(X_test, y_test)]

In [30]:
import xgboost
from numpy import loadtxt
from xgboost import XGBClassifier
# fit model no training data
model = XGBClassifier()
 
from sklearn import tree
model.fit(X_train,y_train)
df_test_output= model.predict(X_test)
import pandas as pd
import numpy as np
np.unique(y_test, return_counts=True)
np.unique(df_test_output, return_counts=True)
 


(array([0, 1], dtype=int64), array([344825,  56643], dtype=int64))

In [31]:
accuracy_score(y_test, df_test_output)

0.9146332958043979

# Accuracy calculation

In [32]:
accuracy = accuracy_score(np.array(y_test).flatten(),df_test_output)
print("Accuracy: %.10f%%" % (accuracy * 100.0))

Accuracy: 91.4633295804%


# Confusion matrix

In [36]:
from sklearn.metrics import confusion_matrix 
from sklearn.metrics import accuracy_score 
from sklearn.metrics import classification_report 
actual = y_test
predicted = df_test_output
results = confusion_matrix(actual, predicted) 
print ('Confusion Matrix :')
print(results) 
 
print (classification_report(actual, predicted))

Confusion Matrix :
[[311613   1060]
 [ 33212  55583]]
              precision    recall  f1-score   support

           0       0.90      1.00      0.95    312673
           1       0.98      0.63      0.76     88795

   micro avg       0.91      0.91      0.91    401468
   macro avg       0.94      0.81      0.86    401468
weighted avg       0.92      0.91      0.91    401468

